In [1]:
import sys
import os

# Get the root directory
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the root directory to the Python path
sys.path.append(root_dir)

In [2]:
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from TINTOlib.refined import REFINED
from kan import *
from tqdm import tqdm


import traceback
import time
import gc
import copy
import traceback
import torch.nn as nn
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import csv
#from torch.optim import LBFGS


if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

cuda


In [3]:
SEED = 381
# SET RANDOM SEED FOR REPRODUCIBILITY
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

# BEST: ACC = 0.733152

In [4]:
folder="data/FOREX_cadjpy-day-High"
x_col=["Bid_Open", "Bid_High", "Bid_Low", "Bid_Close", "Bid_Volume", "Ask_Open", "Ask_High", "Ask_Low", "Ask_Close","Ask_Volume"]
target_col=["class"]

# Functions

### Load Dataset and Images

In [5]:
def load_and_clean(npy_filename, y_filename, x_col, target_col):
    """
    Load the feature array (npy_filename) and label array (y_filename),
    drop rows in the feature array that contain any NaNs, and apply
    the same mask to the label array.
    """
    # Load numpy arrays
    X = np.load(os.path.join(folder, npy_filename))
    y = np.load(os.path.join(folder, y_filename))
    
    # Ensure the number of rows matches between X and y
    if X.shape[0] != y.shape[0]:
        raise ValueError("The number of rows in {} and {} do not match.".format(npy_filename, y_filename))
    
    # Create a boolean mask for rows that do NOT have any NaN values in X
    valid_rows = ~np.isnan(X).any(axis=1)
    #print(valid_rows)
    # Filter both arrays using the valid_rows mask
    X_clean = X[valid_rows]
    y_clean = y[valid_rows]
    
    # Convert arrays to DataFrames
    df_X = pd.DataFrame(X_clean)
    df_y = pd.DataFrame(y_clean)
    df_X.columns = x_col
    df_y.columns = target_col

    #df_y[target_col] = df_y[target_col].astype(int)
    return df_X, df_y

In [6]:
def load_and_preprocess_data(X_train, y_train, X_test, y_test, X_val, y_val, image_model, problem_type, batch_size=32):
    # Add target column to input for IGTD
    X_train_full = X_train.copy()
    X_train_full["target"] = y_train.values

    X_val_full = X_val.copy()
    X_val_full["target"] = y_val.values

    X_test_full = X_test.copy()
    X_test_full["target"] = y_test.values

    # Generate the images if the folder does not exist
    if not os.path.exists(f'{images_folder}/train'):
        #print(X_train_full)
        image_model.fit_transform(X_train_full, f'{images_folder}/train')
        image_model.saveHyperparameters(f'{images_folder}/model.pkl')
    else:
        print("The images are already generated")

    # Load image paths
    imgs_train = pd.read_csv(os.path.join(f'{images_folder}/train', f'{problem_type}.csv'))
    imgs_train["images"] = images_folder + "/train/" + imgs_train["images"]

    if not os.path.exists(f'{images_folder}/val'):
        image_model.transform(X_val_full, f'{images_folder}/val')
    else:
        print("The images are already generated")

    imgs_val = pd.read_csv(os.path.join(f'{images_folder}/val', f'{problem_type}.csv'))
    imgs_val["images"] = images_folder + "/val/" + imgs_val["images"]

    if not os.path.exists(f'{images_folder}/test'):
        image_model.transform(X_test_full, f'{images_folder}/test')
    else:
        print("The images are already generated")

    imgs_test = pd.read_csv(os.path.join(f'{images_folder}/test', f'{problem_type}.csv'))
    imgs_test["images"] = images_folder + "/test/" + imgs_test["images"]

    # Image data
    X_train_img = np.array([cv2.imread(img) for img in imgs_train["images"]])
    X_val_img = np.array([cv2.imread(img) for img in imgs_val["images"]])
    X_test_img = np.array([cv2.imread(img) for img in imgs_test["images"]])

    # Create a MinMaxScaler object
    scaler = MinMaxScaler()

    # Scale numerical data
    X_train_num = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    X_val_num = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)
    X_test_num = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    attributes = len(X_train_num.columns)
    height, width, channels = X_train_img[0].shape
    imgs_shape = (channels, height, width)

    print("Images shape: ", imgs_shape)
    print("Attributes: ", attributes)

    # Convert data to PyTorch tensors
    X_train_num_tensor = torch.as_tensor(X_train_num.values, dtype=torch.float32)
    X_val_num_tensor = torch.as_tensor(X_val_num.values, dtype=torch.float32)
    X_test_num_tensor = torch.as_tensor(X_test_num.values, dtype=torch.float32)
    X_train_img_tensor = torch.as_tensor(X_train_img, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
    X_val_img_tensor = torch.as_tensor(X_val_img, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
    X_test_img_tensor = torch.as_tensor(X_test_img, dtype=torch.float32).permute(0, 3, 1, 2) / 255.0
    y_train_tensor = torch.as_tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
    y_val_tensor = torch.as_tensor(y_val.values, dtype=torch.float32).reshape(-1, 1)
    y_test_tensor = torch.as_tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)

    # Create DataLoaders
    train_dataset = TensorDataset(X_train_num_tensor, X_train_img_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_num_tensor, X_val_img_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_num_tensor, X_test_img_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

    return train_loader, val_loader, test_loader, attributes, imgs_shape

In [7]:
def combine_loader(loader):
    """
    Combines all batches from a DataLoader into three tensors.
    Assumes each batch is a tuple: (mlp_tensor, img_tensor, target_tensor)
    """
    mlp_list, img_list, target_list = [], [], []
    for mlp, img, target in loader:
        mlp_list.append(mlp)
        img_list.append(img)
        target_list.append(target)
    return torch.cat(mlp_list, dim=0), torch.cat(img_list, dim=0), torch.cat(target_list, dim=0)

### Functions for KAN

In [8]:
dtype = torch.get_default_dtype()
min_expected = 0.
max_expected = 1.

In [42]:
folder_path=f"{dataset_name}_{name}_info"

def store_model_info_cnn(folder_path, model_state, train_loss, val_loss, config_name):
    # Ensure the main folder exists
    os.makedirs(folder_path, exist_ok=True)

    # Create a unique subfolder name based on the configuration
    config_path = os.path.join(folder_path, config_name)
    os.makedirs(config_path, exist_ok=True)

    # Save model state
    model_path = os.path.join(config_path, "model.pt")
    torch.save(model_state, model_path)

    # Save training and validation loss curves
    np.save(os.path.join(config_path, "train_loss.npy"), np.array(train_loss))
    np.save(os.path.join(config_path, "val_loss.npy"), np.array(val_loss))

    print(f"📁 Saved model and losses to: {config_path}")


### Functions for CNN

In [9]:
def fit_cnn_only_model(model, dataset, steps=100, lr=1.0, loss_fn=None, batch=-1, opt="LBFGS", metrics=None, display_metrics=None):
    """
    Trains a CNN-only model using LBFGS.

    Args:
        model: CNN-only PyTorch model.
        dataset: Dictionary with keys: 'train_img', 'train_label', 'val_img', 'val_label'.
        steps: Number of training iterations.
        lr: Learning rate.
        loss_fn: Loss function. Defaults to MSE.
        metrics : a list of metrics (as functions)
            the metrics to be computed in training
        display_metrics : a list of functions
            the metric to be displayed in tqdm progress bar

    Returns:
        results: Dict with lists of train/val losses.
        best_model_state: Best weights based on val loss.
    """
    device = model.device
    if loss_fn is None:
        loss_fn = nn.MSELoss()

    # Optimizer selection
    if opt == "LBFGS":
        optimizer = LBFGS(model.parameters(), lr=lr, history_size=10, 
                          line_search_fn="strong_wolfe", 
                          tolerance_grad=1e-32, 
                          tolerance_change=1e-32, 
                          tolerance_ys=1e-32)
    elif opt == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr)
    else:
        raise ValueError(f"Unsupported optimizer '{opt}'")

    n_train = dataset["train_img"].shape[0]
    n_val = dataset["val_img"].shape[0]
    batch_size = n_train if batch == -1 or batch > n_train else batch

    results = {'train_loss': [], 'val_loss': []}

    if metrics != None:
        for i in range(len(metrics)):
            results[metrics[i].__name__] = []

    best_model_state = None
    best_epoch = -1
    best_metric = 0
    val_metric = 0

    pbar = tqdm(range(steps), desc="Training CNN Only ({opt})", ncols=100)

    for step in pbar:
        train_idx = np.random.choice(n_train, batch_size, replace=False)
        #train_idx = torch.randperm(n_train)[:min(32, n_train)]  # small batch
        x_train = dataset["train_img"][train_idx].to(device)
        y_train = dataset["train_label"][train_idx].to(device)
        if opt == "LBFGS":
            def closure():
                optimizer.zero_grad()
                output = model(0, x_train)
                loss = loss_fn(output, y_train)
                loss.backward()
                return loss
            optimizer.step(closure)
            train_loss = closure().item()

        else:  # AdamW
            optimizer.zero_grad()
            output = model(0, x_train)
            loss = loss_fn(output, y_train)
            loss.backward()
            optimizer.step()
            train_loss = loss.item()

        with torch.no_grad():
            val_output = model(0, dataset["val_img"].to(device))
            val_loss = loss_fn(val_output, dataset["val_label"].to(device)).item()

        results["train_loss"].append(train_loss)
        results["val_loss"].append(val_loss)

        if metrics != None:
            for i in range(len(metrics)):
                results[metrics[i].__name__].append(metrics[i]().item())

        if display_metrics == None:
            pbar.set_description(f"| Train: {train_loss:.4e} | Val: {val_loss:.4e} |")
        else:
            string = ''
            data = ()
            for metric in display_metrics:
                val_metric = results[metric][-1]
                string += f' {metric}: %.2e |'
                try:
                    results[metric]
                except:
                    raise Exception(f'{metric} not recognized')
                data += (results[metric][-1],)
            pbar.set_description(string % data)
        
        if val_metric > best_metric:
            best_epoch = step
            best_metric = val_metric
            best_model_state = copy.deepcopy(model.state_dict())

    print(f"✅ Best validation Acc: {best_metric:.4e} at {best_epoch} epoch")
    return best_model_state, results, best_epoch

In [10]:
def build_custom_cnn_model(cnn_blocks, dense_layers, imgs_shape, device='cuda'):
    class CustomCNNModel(nn.Module):
        def __init__(self):
            super(CustomCNNModel, self).__init__()
            self.device = device

            cnn_layers = []
            in_channels = imgs_shape[0]
            out_channels = 16
            cnn_blocks_list = [10, 7, 5]
            size_layer_norm = cnn_blocks_list[cnn_blocks-1]
            
            f_layer_size = 10 - cnn_blocks*2
            for i in range(cnn_blocks):
                cnn_layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=2))
                
                if i < cnn_blocks - 1:
                    cnn_layers.append(nn.BatchNorm2d(out_channels))
                    cnn_layers.append(nn.ReLU())
                    cnn_layers.append(nn.MaxPool2d(2))
                else:
                    # Last block: LayerNorm + Sigmoid + Flatten
                    cnn_layers.append(nn.LayerNorm([out_channels, size_layer_norm, size_layer_norm]))
                    cnn_layers.append(nn.Sigmoid())
                    cnn_layers.append(nn.Flatten())
                in_channels = out_channels
                out_channels *= 2

            self.cnn_branch = nn.Sequential(*cnn_layers).to(device)
            self.flat_size = self._get_flat_size(imgs_shape)

            # Dense (FC) layers
            fc_layers = []
            input_dim = self.flat_size
            for i in range(dense_layers - 1):
                fc_layers.append(nn.Linear(int(input_dim), int(input_dim // 2)))
                fc_layers.append(nn.ReLU())
                input_dim = input_dim // 2
            fc_layers.append(nn.Linear(int(input_dim), 1))

            self.fc = nn.Sequential(*fc_layers).to(device)

        def _get_flat_size(self, imgs_shape):
            dummy_input = torch.zeros(1, *imgs_shape, device=self.device)
            x = self.cnn_branch(dummy_input)
            return x.shape[1]

        def forward(self, num_input, img_input):
            img_input = img_input.to(self.device)
            features = self.cnn_branch(img_input)
            output = self.fc(features)
            return output

    return CustomCNNModel()

In [47]:
def grid_search_cnn(dataset, steps=110, opt='LBFGS', batch=-1, filename=None,
                    cnn_blocks_options = list(range(1, 4)),
                    dense_layers_options = list(range(1, 4)),
                    margin_metric=0.73
                   ):

    top_models = []  # list of dicts: each will have 'loss', 'config', 'state_dict'

    for cnn_blocks in cnn_blocks_options:
        for dense_layers in dense_layers_options:
            print(f"Testing cnn_blocks={cnn_blocks}, dense_layers={dense_layers}")
            model = build_custom_cnn_model(cnn_blocks, dense_layers, imgs_shape)
            def train_acc_cnn():
                rounded = torch.round(torch.round(model(0, dataset['train_img'])[:,0]))
                clipped = torch.clamp(rounded, min=min_expected, max=max_expected)
                return torch.mean((clipped == dataset['train_label'][:,0]).type(dtype))
            
            def val_acc_cnn():
                rounded = torch.round(torch.round(model(0, dataset['val_img'])[:,0]))
                clipped = torch.clamp(rounded, min=min_expected, max=max_expected)
                return torch.mean((clipped == dataset['val_label'][:,0]).type(dtype))
            # Custom training
            model_state, results, best_epoch = fit_cnn_only_model(model, dataset, steps=steps, opt=opt, batch=batch, 
                                                        metrics=(train_acc_cnn, val_acc_cnn), display_metrics=['train_acc_cnn', 'val_acc_cnn'])
            model.load_state_dict(model_state)

            clipped = torch.clamp(torch.round(model(0, dataset['test_img'])[:,0]), min=min_expected, max=max_expected)
            avg_acc = torch.mean((clipped == dataset['test_label'][:,0]).type(dtype))
            print(avg_acc)
            # Save this model info
            top_models.append({
                "avg_acc": avg_acc,
                "config": {"cnn_blocks": cnn_blocks, "dense_layers": dense_layers},
                #"state_dict": copy.deepcopy(model_state),
                "best_epoch": best_epoch
            })
            if avg_acc.item() > margin_metric:
                config_name = f"avg_acc{avg_acc:.5f}_cnn{cnn_blocks}_d{dense_layers}_e{best_epoch}"
                store_model_info_cnn(folder_path, model_state, results['train_acc_cnn'], results['val_acc_cnn'], config_name)
                append_row_to_csv(filename, cnn_blocks, dense_layers, '_', avg_acc.item(), best_epoch)
            del model
            torch.cuda.empty_cache()
            gc.collect()
            # Keep only top 5
            top_models = sorted(top_models, key=lambda x: x["avg_acc"], reverse=True)[:5]

    print("\n✅ Top 5 Configurations:")
    for i, entry in enumerate(top_models):
        cfg = entry["config"]
        print(f"{i+1}. cnn_blocks={cfg['cnn_blocks']}, dense_layers={cfg['dense_layers']} | best_epoch={entry['best_epoch']} | acc={entry['avg_acc']:.5f}")

    #return top_models

### Hybrid Functions

In [28]:
def fit_hybrid_dataloaders(model, 
                           dataset,
                           opt="AdamW", 
                           steps=100, 
                           log=1, 
                           lamb=0., 
                           lamb_l1=1., 
                           lamb_entropy=2., 
                           lamb_coef=0., 
                           lamb_coefdiff=0., 
                           update_grid=True, 
                           grid_update_num=10, 
                           loss_fn=None, 
                           lr=1., 
                           start_grid_update_step=-1, 
                           stop_grid_update_step=50, 
                           batch=-1,
                           metrics=None, 
                           save_fig=False, 
                           in_vars=None, 
                           out_vars=None, 
                           beta=3, 
                           save_fig_freq=1, 
                           img_folder='./video', 
                           singularity_avoiding=False, 
                           y_th=1000., 
                           reg_metric='edge_forward_spline_n', 
                           display_metrics=None,
                           sum_f_reg=True):
    """
    Trains the hybrid model (with a KAN branch and a CNN branch) using a steps-based loop
    adapted from KAN.fit(), with grid updates and regularization.
    
    Instead of a single dataset dict, this function accepts three DataLoaders:
        - train_loader: provides (mlp, img, target) for training
        - val_loader: provides (mlp, img, target) for evaluation during training
        - test_loader: provides (mlp, img, target) for validation

    Internally, the function combines each loader into a dataset dictionary.
    
    Returns:
        results: dictionary containing training loss, evaluation loss, regularization values,
                 and any additional metrics recorded during training.
    """
    #device = next(model.parameters()).device

    # Warn if regularization is requested but model's internal flag isn't enabled.
    if lamb > 0. and not getattr(model.m_kan, "save_act", False):
        print("setting lamb=0. If you want to set lamb > 0, set model.m_kan.save_act=True")
    
    # Disable symbolic processing for training if applicable (KAN internal logic)
    if hasattr(model.m_kan, "disable_symbolic_in_fit"):
        old_save_act, old_symbolic_enabled = model.m_kan.disable_symbolic_in_fit(lamb)
        f_old_save_act, f_old_symbolic_enabled = model.final_kan.disable_symbolic_in_fit(lamb)
    else:
        old_save_act, old_symbolic_enabled = None, None

    pbar = tqdm(range(steps), desc='Training', ncols=100)

    # Default loss function (mean squared error) if not provided
    if loss_fn is None:
        loss_fn = lambda x, y: torch.mean((x - y) ** 2)

    # Determine grid update frequency
    grid_update_freq = int(stop_grid_update_step / grid_update_num) if grid_update_num > 0 else 1

    # Determine total number of training examples
    n_train = dataset["train_input"].shape[0]
    n_eval  = dataset["val_input"].shape[0]  # using val set for evaluation during training
    batch_size = n_train if batch == -1 or batch > n_train else batch

    # Set up optimizer: choose between Adam and LBFGS (removed tolerance_ys)
    if opt == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr)
    elif opt == "LBFGS":        
        optimizer = LBFGS(model.parameters(), lr=lr, history_size=10, 
                          line_search_fn="strong_wolfe", 
                          tolerance_grad=1e-32, 
                          tolerance_change=1e-32, 
                          tolerance_ys=1e-32)
    else:
        raise ValueError("Optimizer not recognized. Use 'Adam' or 'LBFGS'.")

    # Prepare results dictionary.
    results = {'train_loss': [], 'eval_loss': [], 'reg': []}
    
    if metrics is not None:
        for metric in metrics:
            results[metric.__name__] = []

    best_model_state = None
    best_epoch = -1
    best_metric = 0
    val_metric = 0

    for step in pbar:
        # Randomly sample indices for a mini-batch from the training set.
        train_indices = np.random.choice(n_train, batch_size, replace=False)
        # Use full evaluation set for evaluation; you can also sample if desired.
        eval_indices = np.arange(n_eval)
        
        cached_loss = {}
        # Closure for LBFGS
        def closure():
            optimizer.zero_grad()
            mlp_batch = dataset["train_input"][train_indices]
            img_batch = dataset["train_img"][train_indices]
            target_batch = dataset["train_label"][train_indices]
            outputs = model(mlp_batch, img_batch)
            train_loss = loss_fn(outputs, target_batch)
            # Compute regularization term if enabled.
            if hasattr(model.m_kan, "save_act") and model.m_kan.save_act:
                if reg_metric == 'edge_backward':
                    model.m_kan.attribute()
                    model.final_kan.attribute()
                if reg_metric == 'node_backward':
                    model.m_kan.node_attribute()
                    model.final_kan.node_attribute()
                reg_val_inner = model.m_kan.get_reg(reg_metric, lamb_l1, lamb_entropy, lamb_coef, lamb_coefdiff)
                if sum_f_reg:
                    reg_val_inner += model.final_kan.get_reg(reg_metric, lamb_l1, lamb_entropy, lamb_coef, lamb_coefdiff)
            else:
                reg_val_inner = torch.tensor(0., device=device)
            loss_val_inner = train_loss + lamb * reg_val_inner
            loss_val_inner.backward()
            cached_loss['loss'] = loss_val_inner.detach()
            cached_loss['reg'] = reg_val_inner.detach()
            return loss_val_inner

        # Perform grid update if applicable.
        if (step % grid_update_freq == 0 and step < stop_grid_update_step 
            and update_grid and step >= start_grid_update_step):
            
            mlp_batch = dataset['train_input'][train_indices]
            cnn_batch = dataset['train_img'][train_indices]
            
            model.m_kan.update_grid(mlp_batch)
            #cnn_output = model.cnn_branch(cnn_batch)  # Process image input
            concatenated = model.get_concat_output(mlp_batch, cnn_batch)

            model.final_kan.update_grid(concatenated)

        # Perform an optimizer step.
        if opt == "LBFGS":
            optimizer.step(closure)
            loss_val = cached_loss['loss']
            reg_val = cached_loss['reg']
        else:  # AdamW branch
            optimizer.zero_grad()
            mlp_batch = dataset["train_input"][train_indices]
            img_batch = dataset["train_img"][train_indices]
            target_batch = dataset["train_label"][train_indices]
            outputs = model(mlp_batch, img_batch)
            train_loss = loss_fn(outputs, target_batch)
            if hasattr(model.m_kan, "save_act") and model.m_kan.save_act:
                if reg_metric == 'edge_backward':
                    model.m_kan.attribute()
                    model.final_kan.attribute()
                if reg_metric == 'node_backward':
                    model.m_kan.node_attribute()
                    model.final_kan.node_attribute()
                reg_val = model.m_kan.get_reg(reg_metric, lamb_l1, lamb_entropy, lamb_coef, lamb_coefdiff)
                if sum_f_reg:
                    reg_val = reg_val + model.final_kan.get_reg(reg_metric, lamb_l1, lamb_entropy, lamb_coef, lamb_coefdiff)
            else:
                reg_val = torch.tensor(0., device=device)
            loss_val = train_loss + lamb * reg_val
            loss_val.backward()
            optimizer.step()


        with torch.no_grad():
            mlp_eval = dataset["val_input"][eval_indices]
            img_eval = dataset["val_img"][eval_indices]
            target_eval = dataset["val_label"][eval_indices]
            eval_loss = loss_fn(model(mlp_eval, img_eval), target_eval)

        # Record results (using square-root of loss similar to KAN.fit)
        results['eval_loss'].append(torch.sqrt(eval_loss.detach()).item())
        results['reg'].append(reg_val.detach().item())

        if metrics is not None:
            for metric in metrics:
                # Here, we assume each metric returns a tensor.
                results[metric.__name__].append(metric().item())


        # Update progress bar.
        if display_metrics is None:
            pbar.set_description("| train_loss: %.2e | eval_loss: %.2e | reg: %.2e |" %
                                 (torch.sqrt(loss_val.detach()).item(),
                                  torch.sqrt(eval_loss.detach()).item(),
                                  reg_val.detach().item()))
        else:
            string = ''
            data = ()
            for metric in display_metrics:
                val_metric = results[metric][-1]
                string += f' {metric}: %.2e |'
                try:
                    results[metric]
                except:
                    raise Exception(f'{metric} not recognized')
                data += (results[metric][-1],)
            pbar.set_description(string % data)

        if val_metric > best_metric:
            best_epoch = step
            best_metric = val_metric
            best_model_state = copy.deepcopy(model.state_dict())

        # Optionally save a figure snapshot.
        if save_fig and step % save_fig_freq == 0:
            save_act_backup = getattr(model.m_kan, "save_act", False)
            model.m_kan.save_act = True
            model.plot(folder=img_folder, in_vars=in_vars, out_vars=out_vars, title=f"Step {step}", beta=beta)
            plt.savefig(os.path.join(img_folder, f"{step}.jpg"), bbox_inches='tight', dpi=200)
            plt.close()
            model.m_kan.save_act = save_act_backup

    # Restore original settings if applicable.
    if old_symbolic_enabled is not None:
        model.m_kan.symbolic_enabled = old_symbolic_enabled
    if hasattr(model.m_kan, "log_history"):
        model.m_kan.log_history('fit')
    print(f"✅ Best validation Accuracy: {best_metric:.4e} at {best_epoch} epoch")
    return best_model_state, results, best_epoch

In [75]:
def try_create_model(model_class, attributes, imgs_shape, kan_neurons, kan_grid, cnn_bottleneck_dim=-1, alpha=-1, hidden_dim=-1, embed_dim=-1, num_heads=-1):
    try:
        model = model_class(attributes, imgs_shape, kan_neurons, kan_grid, cnn_bottleneck_dim=cnn_bottleneck_dim, alpha=alpha, hidden_dim=hidden_dim, embed_dim=embed_dim, num_heads=num_heads)
        
        # Test the model with a sample input
        num_input = torch.randn(4, attributes)
        img_input = torch.randn(4, *imgs_shape)
        output = model(num_input, img_input)
        
        print(f"Successfully created and tested {model_class.__name__}")
        
        return model
    except Exception as e:
        print(f"Error creating or testing {model_class.__name__}:")
        traceback.print_exc()
        return None

In [78]:
def grid_search_kan_hybrid(dataset, model_class, filename=None, hidden_neuron_options=list(range(1, 3)), max_steps=30, margin_metric=0.739,
                           grid_options = list(range(3, 9)), lamb_options = [1e-01, 1e-02, 1e-03, 1e-04, 1e-05, 1e-06, 1e-07],
                          cnn_bottleneck_dim=-1, alpha=-1, hidden_dim=-1, embed_dim=-1, num_heads=-1,
                          model_name='Hybrid'):

    for hidden in hidden_neuron_options:
        for grid in grid_options:
            for lamb in lamb_options:
                print(f"Testing {hidden} hidden neurons, lamb={lamb}, grid={grid}")
                model = try_create_model(model_class, attributes, imgs_shape, kan_neurons=hidden, kan_grid=grid,
                                        cnn_bottleneck_dim=cnn_bottleneck_dim, alpha=alpha, hidden_dim=hidden_dim, embed_dim=embed_dim, num_heads=num_heads)  # Attempt to create Model3
                def train_acc_hybrid():
                    rounded = torch.round(torch.round(model(dataset['train_input'], dataset['train_img'])[:,0]))
                    clipped = torch.clamp(rounded, min=min_expected, max=max_expected)
                    return torch.mean((clipped == dataset['train_label'][:,0]).type(dtype))
                
                def val_acc_hybrid():
                    rounded = torch.round(torch.round(model(dataset['val_input'], dataset['val_img'])[:,0]))
                    clipped = torch.clamp(rounded, min=min_expected, max=max_expected)
                    return torch.mean((clipped == dataset['val_label'][:,0]).type(dtype))

                # Custom training
                model_state, results, best_epoch = fit_hybrid_dataloaders(model, dataset, opt="LBFGS", steps=max_steps, lamb=lamb,
                                                                   metrics=(train_acc_hybrid, val_acc_hybrid), display_metrics=['train_acc_hybrid', 'val_acc_hybrid'])
                model.load_state_dict(model_state)

                # Evaluate
                clipped = torch.clamp(torch.round(model(dataset['test_input'], dataset['test_img'])[:,0]), min=min_expected, max=max_expected)
                avg_acc = torch.mean((clipped == dataset['test_label'][:,0]).type(dtype)).item()
                print(f'avg_acc {avg_acc}')
                
                if avg_acc > margin_metric:
                    config_name = f"{model_name}_avg_acc{avg_acc:.5f}_h{hidden}_g{grid}_l{lamb}_e{best_epoch}"
                    store_model_info_cnn(folder_path, model_state, results['train_acc_hybrid'], results['val_acc_hybrid'], config_name)
                    append_row_to_csv(filename, hidden, grid, lamb, avg_acc, best_epoch)
                # Save this model info
                # top_models.append({
                #     "avg_acc": avg_acc,
                #     "config": {"hidden": hidden, "grid": grid, "lamb": lamb},
                #     #"state_dict": copy.deepcopy(model_state),
                #     "best_epoch": best_epoch
                # })
                del model
                torch.cuda.empty_cache()
                gc.collect()
        # Keep only top 5
    #     top_models = sorted(top_models, key=lambda x: x["avg_acc"], reverse=True)[:5]

    # print("\n✅ Top 5 Configurations:")
    # for i, entry in enumerate(top_models):
    #     cfg = entry["config"]
    #     print(f"{i+1}. width=[{attributes}, {cfg['hidden']}, 1], grid={cfg['grid']}, lamb={cfg['lamb']}| best_epoch={entry['best_epoch']} | avg_acc={entry['avg_acc']:.5f}")


# Set Files Name

In [31]:
def create_csv_with_header(filename, header=['kan_neurons', 'kan_grid', 'lamb', 'ACC', 'Best_Epoch']):
    """Creates a CSV file with a given header."""
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)

In [32]:
def append_row_to_csv(filename, kan_neurons, kan_grid, lamb, acc, best_epoch):
    row = [kan_neurons, kan_grid, lamb, acc, best_epoch]
    """Appends a single row to an existing CSV file."""
    if not os.path.isfile(filename):
        raise FileNotFoundError(f"{filename} does not exist. Please create the file first with a header.")
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

# Load Dataset and Images

In [33]:
X_train, y_train = load_and_clean('N_train.npy', 'y_train.npy',x_col, target_col)
X_test, y_test   = load_and_clean('N_test.npy',  'y_test.npy', x_col, target_col)
X_val, y_val     = load_and_clean('N_val.npy',   'y_val.npy', x_col, target_col)

In [34]:
# Get the shape of the dataframe
num_columns = X_train.shape[1]

# Calculate number of columns - 1
columns_minus_one = num_columns - 1

# Calculate the square root for image size
image_size = math.ceil(math.sqrt(num_columns))
print(image_size)

4


In [35]:
dataset_name = 'FOREX_cadjpy-day-High'
#Select the model and the parameters
problem_type = "supervised"
image_model = REFINED(problem=problem_type, zoom=2, random_seed=SEED)
name = f"Refined"

#Define the dataset path and the folder where the images will be saved
images_folder = f"HyNNImages/Classification/{dataset_name}/images_{dataset_name}_{name}"

In [36]:
train_loader, val_loader, test_loader, attributes, imgs_shape = load_and_preprocess_data(
    X_train, y_train, X_test, y_test, X_val, y_val,
    image_model=image_model,
    problem_type=problem_type,
    batch_size=16
)

The images are already generated
The images are already generated
The images are already generated
Images shape:  (3, 8, 8)
Attributes:  10


In [37]:
# Combine dataloaders into tensors.
train_mlp, train_img, train_target = combine_loader(train_loader)
val_mlp, val_img, val_target = combine_loader(val_loader)
test_mlp, test_img, test_target = combine_loader(test_loader)

dataset = {
    "train_input": train_mlp.to(device),
    "train_img": train_img.to(device),
    "train_label": train_target.to(device),
    "val_input": val_mlp.to(device),
    "val_img": val_img.to(device),
    "val_label": val_target.to(device),
    "test_input": test_mlp.to(device),
    "test_img": test_img.to(device),
    "test_label": test_target.to(device),
}

In [38]:
# Print the shapes of the tensors
print("Train data shape:", dataset['train_input'].shape)
print("Train target shape:", dataset['train_label'].shape)
print("Test data shape:", dataset['test_input'].shape)
print("Test target shape:", dataset['test_label'].shape)
print("Validation data shape:", dataset['val_input'].shape)
print("Validation target shape:", dataset['val_label'].shape)

Train data shape: torch.Size([1173, 10])
Train target shape: torch.Size([1173, 1])
Test data shape: torch.Size([367, 10])
Test target shape: torch.Size([367, 1])
Validation data shape: torch.Size([294, 10])
Validation target shape: torch.Size([294, 1])


In [39]:
filename_1=f'{dataset_name}_refined_grid_search_kan.csv'
filename_2=f'{dataset_name}_refined_grid_search_cnn.csv'
filename_3=f'{dataset_name}_refined_grid_search_hybrid3.csv'
filename_4=f'{dataset_name}_refined_grid_search_hybrid4.csv'

# Grid Search for CNN

In [40]:
create_csv_with_header(filename_2, header=['cnn_blocks', 'dense_layers', '_', 'Acc', 'Best_Epoch'])

In [50]:
grid_search_cnn(dataset, steps=65, filename=filename_2, cnn_blocks_options=[1,3])

Testing cnn_blocks=1, dense_layers=1


 train_acc_cnn: 7.43e-01 | val_acc_cnn: 7.14e-01 |: 100%|███████████| 65/65 [00:22<00:00,  2.86it/s]


✅ Best validation Acc: 7.2789e-01 at 1 epoch
tensor(0.7139, device='cuda:0')
Testing cnn_blocks=1, dense_layers=2


 train_acc_cnn: 7.42e-01 | val_acc_cnn: 7.07e-01 |: 100%|███████████| 65/65 [00:12<00:00,  5.00it/s]


✅ Best validation Acc: 7.2789e-01 at 1 epoch
tensor(0.7057, device='cuda:0')
Testing cnn_blocks=1, dense_layers=3


...

✅ Best validation Acc: 7.2109e-01 at 8 epoch
tensor(0.7357, device='cuda:0')
📁 Saved model and losses to: FOREX_cadjpy-day-High_Refined_info\avg_acc0.73569_cnn3_d1_e8
Testing cnn_blocks=3, dense_layers=2


 train_acc_cnn: 7.72e-01 | val_acc_cnn: 7.11e-01 |: 100%|███████████| 65/65 [00:12<00:00,  5.27it/s]


✅ Best validation Acc: 7.3129e-01 at 47 epoch
tensor(0.7302, device='cuda:0')
📁 Saved model and losses to: FOREX_cadjpy-day-High_Refined_info\avg_acc0.73025_cnn3_d2_e47
Testing cnn_blocks=3, dense_layers=3


 train_acc_cnn: 8.00e-01 | val_acc_cnn: 6.70e-01 |: 100%|███████████| 65/65 [00:13<00:00,  4.84it/s]

✅ Best validation Acc: 7.2109e-01 at 2 epoch
tensor(0.7275, device='cuda:0')

✅ Top 5 Configurations:
1. cnn_blocks=3, dense_layers=1 | best_epoch=8 | acc=0.73569
2. cnn_blocks=3, dense_layers=2 | best_epoch=47 | acc=0.73025
3. cnn_blocks=3, dense_layers=3 | best_epoch=2 | acc=0.72752
4. cnn_blocks=1, dense_layers=1 | best_epoch=1 | acc=0.71390
5. cnn_blocks=1, dense_layers=3 | best_epoch=6 | acc=0.71390


In [51]:
grid_search_cnn(dataset, steps=65, filename=filename_2, cnn_blocks_options=[1,3])

Testing cnn_blocks=1, dense_layers=1


 train_acc_cnn: 7.39e-01 | val_acc_cnn: 7.04e-01 |: 100%|███████████| 65/65 [00:04<00:00, 13.16it/s]


✅ Best validation Acc: 7.3810e-01 at 1 epoch
tensor(0.7221, device='cuda:0')
Testing cnn_blocks=1, dense_layers=2


 train_acc_cnn: 7.49e-01 | val_acc_cnn: 7.01e-01 |: 100%|███████████| 65/65 [00:07<00:00,  8.90it/s]


✅ Best validation Acc: 7.2789e-01 at 2 epoch
tensor(0.7084, device='cuda:0')
Testing cnn_blocks=1, dense_layers=3


...

✅ Best validation Acc: 7.2109e-01 at 15 epoch
tensor(0.7193, device='cuda:0')
Testing cnn_blocks=3, dense_layers=2


 train_acc_cnn: 7.68e-01 | val_acc_cnn: 7.04e-01 |: 100%|███████████| 65/65 [00:11<00:00,  5.47it/s]


✅ Best validation Acc: 7.3810e-01 at 1 epoch
tensor(0.7084, device='cuda:0')
Testing cnn_blocks=3, dense_layers=3


 train_acc_cnn: 7.92e-01 | val_acc_cnn: 6.67e-01 |: 100%|███████████| 65/65 [00:13<00:00,  4.86it/s]

✅ Best validation Acc: 7.3129e-01 at 15 epoch
tensor(0.7357, device='cuda:0')
📁 Saved model and losses to: FOREX_cadjpy-day-High_Refined_info\avg_acc0.73569_cnn3_d3_e15

✅ Top 5 Configurations:
1. cnn_blocks=3, dense_layers=3 | best_epoch=15 | acc=0.73569
2. cnn_blocks=1, dense_layers=1 | best_epoch=1 | acc=0.72207
3. cnn_blocks=1, dense_layers=3 | best_epoch=6 | acc=0.72207
4. cnn_blocks=3, dense_layers=1 | best_epoch=15 | acc=0.71935
5. cnn_blocks=1, dense_layers=2 | best_epoch=2 | acc=0.70845


# Grid Search for Hibrid Model3

In [84]:
class Model3(nn.Module):
    def __init__(self, attributes, imgs_shape, kan_neurons, kan_grid, cnn_bottleneck_dim=-1, alpha=-1, hidden_dim=-1, embed_dim=-1, num_heads=-1, device=device):
        super(Model3, self).__init__()

        self.cnn_branch = nn.Sequential(
            nn.Conv2d(imgs_shape[0], 16, kernel_size=3, padding=2),     # out: 16 x 9 x 9
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),                                            # out: 16 x 4 x 4
            
            nn.Conv2d(16, 32, kernel_size=3, padding=2),                # out: 32 x 5 x 5
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),                                            # 32 x 2 x 2
            
            nn.Conv2d(32, 64, kernel_size=3, padding=2),
            nn.LayerNorm([64, 5, 5]),
            nn.Sigmoid(),
            nn.Flatten()
        ).to(device)

        # Middle KAN layers
        self.m_kan = KAN(
            width=[attributes, kan_neurons],
            grid=kan_grid,
            k=3,
            seed=SEED,
            device=device
        )
        # Calculate the size of the flattened output
        self.flat_size = self._get_flat_size(imgs_shape)
        # Final MLP layers
        self.final_kan = KAN(
            width=[self.flat_size + kan_neurons, 1],
            grid=kan_grid,
            k=3,
            device=device
        )
        
        self.device = device

    def _get_flat_size(self, imgs_shape):
        # Forward pass with dummy input to calculate flat size
        dummy_input = torch.zeros(4, *imgs_shape, device=device)
        x = self.cnn_branch(dummy_input)
        return x.size(1)

    def get_concat_output(self, mlp_input, cnn_input):
        # Ensure inputs are moved to the correct device
        kan_input = mlp_input.to(self.device)
        cnn_input = cnn_input.to(self.device)
        
        cnn_output = self.cnn_branch(cnn_input)  # Process image input
        kan_output = self.m_kan(kan_input)  # Process numerical input
        
        return torch.cat((kan_output, cnn_output), dim=1)

    
    def forward(self, mlp_input, cnn_input):
        concat_output = self.get_concat_output(mlp_input, cnn_input)
        return self.final_kan(concat_output)

## Grid Search 3

In [33]:
#create_csv_with_header(filename_3)
3	7
5	7
8	4
5	8
5	8
7	5
7	6
7	8
8	8
3	5
3	6

hidden_neuron_options=list(range(1, 3)), max_steps=30,
                           grid_options = list(range(3, 9)), lamb_options = [1e-01, 1e-02, 1e-03, 1e-04, 1e-05, 1e-06, 1e-07]


In [85]:
grid_search_kan_hybrid(dataset, Model3, filename_3, hidden_neuron_options=[7,3,5,3,5,3], max_steps=10,
                           grid_options=[7,3,7,4,7], lamb_options = [1e-03, 1e-04, 1e-05, 1e-06])

Testing 7 hidden neurons, lamb=0.001, grid=7
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model3


 train_acc_hybrid: 7.38e-01 | val_acc_hybrid: 7.01e-01 |: 100%|█████| 10/10 [00:29<00:00,  2.99s/it]


saving model version 0.1
✅ Best validation Accuracy: 7.3810e-01 at 1 epoch
avg_acc 0.7193460464477539
Testing 7 hidden neurons, lamb=0.0001, grid=7
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model3


 train_acc_hybrid: 7.62e-01 | val_acc_hybrid: 7.14e-01 |: 100%|█████| 10/10 [00:28<00:00,  2.87s/it]


saving model version 0.1
✅ Best validation Accuracy: 7.1769e-01 at 1 epoch
avg_acc 0.7193460464477539
Testing 7 hidden neurons, lamb=1e-05, grid=7
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model3


...

saving model version 0.1
✅ Best validation Accuracy: 7.4490e-01 at 1 epoch
avg_acc 0.7193460464477539
Testing 3 hidden neurons, lamb=1e-05, grid=7
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model3


 train_acc_hybrid: 7.60e-01 | val_acc_hybrid: 7.18e-01 |: 100%|█████| 10/10 [00:26<00:00,  2.70s/it]


saving model version 0.1
✅ Best validation Accuracy: 7.2449e-01 at 2 epoch
avg_acc 0.7411444187164307
📁 Saved model and losses to: FOREX_cadjpy-day-High_Refined_info\Hybrid_avg_acc0.74114_h3_g7_l1e-05_e2
Testing 3 hidden neurons, lamb=1e-06, grid=7
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model3


 train_acc_hybrid: 7.58e-01 | val_acc_hybrid: 7.24e-01 |: 100%|█████| 10/10 [00:27<00:00,  2.78s/it]


saving model version 0.1
✅ Best validation Accuracy: 7.2449e-01 at 0 epoch
avg_acc 0.7084468603134155


In [86]:
class Model4_2(nn.Module):
    def __init__(self, attributes, imgs_shape, kan_neurons, kan_grid, cnn_bottleneck_dim=-1, alpha=-1, hidden_dim=-1, embed_dim=-1, num_heads=-1, device=device):
        super(Model4_2, self).__init__()
        # CNN branch
        self.cnn_branch = nn.Sequential(
            nn.Conv2d(imgs_shape[0], 16, kernel_size=3, padding=2),     # out: 16 x 9 x 9
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),                                            # out: 16 x 4 x 4
            
            nn.Conv2d(16, 32, kernel_size=3, padding=2),                # out: 32 x 5 x 5
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),                                            # 32 x 2 x 2
            
            nn.Conv2d(32, 64, kernel_size=3, padding=2),
            nn.LayerNorm([64, 5, 5]),
            nn.Sigmoid(),
            nn.Flatten()
        ).to(device)

        # Final KAN layers
        self.m_kan = KAN(
            width=[attributes, kan_neurons],
            grid=kan_grid,
            k=3,
            seed=SEED,
            device=device
        )

        # Calculate the size of the flattened output
        self.flat_size = self._get_flat_size(imgs_shape)

        # Final MLP layers
        self.final_kan = KAN(
            width=[self.flat_size + kan_neurons, 1],
            grid=kan_grid,
            k=3,
            seed=SEED,
            device=device
        )

        self.device = device
        self.alpha = alpha


    def _get_flat_size(self, imgs_shape):
        # Forward pass with dummy input to calculate flat size
        dummy_input = torch.zeros(4, *imgs_shape, device=device)
        x = self.cnn_branch(dummy_input)
        return x.size(1)

    def get_concat_output(self, mlp_input, cnn_input):
        # Ensure inputs are moved to the correct device
        kan_input = mlp_input.to(self.device)
        cnn_input = cnn_input.to(self.device)
        
        cnn_output = self.cnn_branch(cnn_input)  # Process image input
        cnn_output = cnn_output * self.alpha
        kan_output = self.m_kan(kan_input)  # Process numerical input
        
        return torch.cat((kan_output, cnn_output), dim=1)

    
    def forward(self, mlp_input, cnn_input):
        concat_output = self.get_concat_output(mlp_input, cnn_input)
        return self.final_kan(concat_output)

In [40]:
#create_csv_with_header(filename_4)
width=[10, 3]
grid=7
lamb=1e-06
alpha=0.8

In [89]:
for alpha in [.95,.9,.85,.8,.75,.7,.6,.65,.5,.45,.4,.35,.3,.25,.2,.15,.1,.05,.01]:
    print(f"alpha: {alpha}")
    grid_search_kan_hybrid(dataset, Model4_2, filename=filename_4, hidden_neuron_options=[3,2], max_steps=15, model_name=f'Concat2_a{alpha}_',
                           grid_options=[3, 4], lamb_options = [1e-05, 1e-07], alpha=alpha)


alpha: 0.95
Testing 3 hidden neurons, lamb=1e-05, grid=3
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model4_2


 train_acc_hybrid: 7.68e-01 | val_acc_hybrid: 7.21e-01 |: 100%|█████| 15/15 [00:36<00:00,  2.40s/it]


saving model version 0.1
✅ Best validation Accuracy: 7.3129e-01 at 8 epoch
avg_acc 0.7166212201118469
Testing 3 hidden neurons, lamb=1e-07, grid=3
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model4_2


 train_acc_hybrid: 7.74e-01 | val_acc_hybrid: 7.24e-01 |: 100%|█████| 15/15 [00:33<00:00,  2.26s/it]


saving model version 0.1
✅ Best validation Accuracy: 7.2449e-01 at 9 epoch
avg_acc 0.7193460464477539
Testing 3 hidden neurons, lamb=1e-05, grid=4
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model4_2


...

saving model version 0.1
✅ Best validation Accuracy: 7.1088e-01 at 0 epoch
avg_acc 0.7029972672462463
Testing 2 hidden neurons, lamb=1e-05, grid=4
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model4_2


 train_acc_hybrid: 8.24e-01 | val_acc_hybrid: 6.84e-01 |: 100%|█████| 15/15 [00:21<00:00,  1.43s/it]


saving model version 0.1
✅ Best validation Accuracy: 7.0748e-01 at 1 epoch
avg_acc 0.7138964533805847
Testing 2 hidden neurons, lamb=1e-07, grid=4
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model4_2


 train_acc_hybrid: 8.15e-01 | val_acc_hybrid: 6.67e-01 |: 100%|█████| 15/15 [00:21<00:00,  1.42s/it]

saving model version 0.1
✅ Best validation Accuracy: 7.0408e-01 at 0 epoch
avg_acc 0.7002724409103394


In [90]:
#_h3_g7_l1e-05
for alpha in [.95,.9,.85,.8,.75,.7,.6,.65,.5,.45,.4,.35,.3,.25,.2,.15,.1,.05,.01]:
    print(f"alpha: {alpha}")
    grid_search_kan_hybrid(dataset, Model4_2, filename=filename_4, hidden_neuron_options=[3], max_steps=15, model_name=f'Concat2_a{alpha}_',
                           grid_options=[7], lamb_options = [1e-05], alpha=alpha)

alpha: 0.95
Testing 3 hidden neurons, lamb=1e-05, grid=7
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model4_2


 train_acc_hybrid: 7.74e-01 | val_acc_hybrid: 6.94e-01 |: 100%|█████| 15/15 [00:28<00:00,  1.90s/it]


saving model version 0.1
✅ Best validation Accuracy: 7.2789e-01 at 11 epoch
avg_acc 0.7302452325820923
alpha: 0.9
Testing 3 hidden neurons, lamb=1e-05, grid=7
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model4_2


 train_acc_hybrid: 7.72e-01 | val_acc_hybrid: 6.97e-01 |: 100%|█████| 15/15 [00:28<00:00,  1.88s/it]


saving model version 0.1
✅ Best validation Accuracy: 7.2789e-01 at 9 epoch
avg_acc 0.7275204062461853
alpha: 0.85
Testing 3 hidden neurons, lamb=1e-05, grid=7
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model4_2


...

saving model version 0.1
✅ Best validation Accuracy: 7.2789e-01 at 1 epoch
avg_acc 0.7111716270446777
alpha: 0.05
Testing 3 hidden neurons, lamb=1e-05, grid=7
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model4_2


 train_acc_hybrid: 7.72e-01 | val_acc_hybrid: 6.90e-01 |: 100%|█████| 15/15 [00:29<00:00,  1.94s/it]


saving model version 0.1
✅ Best validation Accuracy: 6.9048e-01 at 1 epoch
avg_acc 0.6975476741790771
alpha: 0.01
Testing 3 hidden neurons, lamb=1e-05, grid=7
checkpoint directory created: ./model
saving model version 0.0
checkpoint directory created: ./model
saving model version 0.0
Successfully created and tested Model4_2


 train_acc_hybrid: 8.06e-01 | val_acc_hybrid: 6.70e-01 |: 100%|█████| 15/15 [00:29<00:00,  1.95s/it]

saving model version 0.1
✅ Best validation Accuracy: 7.2449e-01 at 1 epoch
avg_acc 0.6975476741790771


#